# RUTA

In [4]:
nombre_busqueda = "80JQ_I" #rutas disponibles '80JQ_I' '80JQ_R' '80KH_I' '80KH_R' '80LZ_I' '80LZ_R'

# ARCHIVOS

In [5]:
rutas = "Recorrido Ruta Las Galaxias L80 (1) (2).kmz"
zonificacion = "zonificacion.kmz"
plan_op="POT_VIII_GRAN+CONCEPCIÃ_N_UN80_NORMAL_2024_A1_5.xlsx"
encuesta_origen_destino = "BD_EOD_GranConcepcion.xlsx"
ARCHIVO_RUTAS_KMZ = rutas
ARCHIVO_ZONAS_KMZ = zonificacion
ARCHIVO_NUEVOS = "nuevos_paraderos.xlsx" 

# PAQUETES

In [6]:
import importlib
def asegurar_paquetes(paquetes):
    """Instala automáticamente los paquetes que no estén instalados."""
    for paquete in paquetes:
        try:
            importlib.import_module(paquete)
        except ImportError:
            print(f"📦 Instalando '{paquete}'...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", paquete]
            )


paquetes_necesarios = [
    "simpy",
    "pygame",
    "fastkml",
    "simplekml",
    "fiona",
    "shapely",
    "geopandas",
    'folium',
]

asegurar_paquetes(paquetes_necesarios)


import subprocess
import sys
import re
import zipfile
import math
import random
from math import radians, cos, sin, asin, sqrt
import xml.etree.ElementTree as ET
from typing import List, Tuple

import numpy as np
import pandas as pd
import folium
from bs4 import BeautifulSoup
import simpy
import pygame
import fastkml
import simplekml
import fiona
import shapely
import geopandas

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
import re
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import folium
from bs4 import BeautifulSoup
import random
from shapely.geometry import LineString, Point

# ============================================================
# ================= CONFIGURACIÓN DE USUARIO =================
# ============================================================



# ============================================================
# ===================== HERRAMIENTAS BASE ====================
# ============================================================

def _clean_kml_text(raw_bytes: bytes) -> str:
    """Limpieza profunda de KML para evitar errores de prefijos."""
    text = raw_bytes.decode("utf-8", errors="replace")
    text = re.sub(r'\s+xmlns(:[A-Za-z0-9_-]+)?="[^"]+"', '', text)
    text = re.sub(r'<(/?)\w+:', r'<\1', text) 
    text = re.sub(r'\s\w+:(\w+=")', r' \1', text) 
    return text

def _parse_coords_string(coords_text: str) -> list[tuple[float, float]]:
    coords = []
    if not coords_text: return coords
    for part in coords_text.strip().split():
        comps = part.split(',')
        if len(comps) >= 2:
            try:
                lon, lat = float(comps[0]), float(comps[1])
                coords.append((lat, lon))
            except ValueError: continue
    return coords

def cargar_paraderos_excel(archivo_excel: str, route_id_destino: str) -> pd.DataFrame:
    """Carga paraderos desde Excel, soporta coordenadas pegadas."""
    print(f"📥 Cargando paraderos extra desde: {archivo_excel}...")
    try:
        df_extra = pd.read_excel(archivo_excel)
        
        # Separar coordenadas si vienen pegadas (formato Google Maps)
        if df_extra['lat'].dtype == 'object' and df_extra['lat'].str.contains(',').any():
            print("🔧 Detectado formato pegado de Google Maps. Separando...")
            split_data = df_extra['lat'].astype(str).str.split(',', expand=True)
            df_extra['lat'] = split_data[0]
            df_extra['lon'] = split_data[1]

        # Convertir a números
        df_extra['lat'] = df_extra['lat'].astype(str).str.replace(',', '.', regex=False).apply(pd.to_numeric, errors='coerce')
        df_extra['lon'] = df_extra['lon'].astype(str).str.replace(',', '.', regex=False).apply(pd.to_numeric, errors='coerce')
        
        df_extra = df_extra.dropna(subset=['lat', 'lon'])
        df_extra["route_id"] = route_id_destino
        
        if "nombre_paradero" not in df_extra.columns:
            df_extra["nombre_paradero"] = "Paradero Manual " + df_extra.index.astype(str)
            
        print(f"✅ Se agregaron {len(df_extra)} paraderos manuales a la ruta {route_id_destino}.")
        return df_extra[ ["route_id", "nombre_paradero", "lat", "lon"] ]

    except Exception as e:
        print(f"⚠️ No se pudo cargar el Excel (o no existe): {e}")
        return pd.DataFrame()

# ============================================================
# ==================== CARGAR ZONIFICACIÓN ===================
# ============================================================

def cargar_zonas_kmz(archivo_kmz: str) -> pd.DataFrame:
    print(f"🌍 Cargando zonas desde: {archivo_kmz}...")
    try:
        with zipfile.ZipFile(archivo_kmz, 'r') as kmz:
            kml_file = [f for f in kmz.namelist() if f.lower().endswith('.kml')][0]
            raw = kmz.read(kml_file)
    except Exception as e:
        print(f"❌ Error leyendo zonas: {e}")
        return pd.DataFrame()

    try:
        soup = BeautifulSoup(raw, "lxml-xml")
    except:
        soup = BeautifulSoup(raw, "html.parser")

    zonas = []
    placemarks = soup.find_all(["Placemark", "placemark"])

    for pm in placemarks:
        name_tag = pm.find(["name", "Name"])
        nombre = name_tag.get_text(strip=True) if name_tag else ""
        zona_val = macrozona_val = None

        desc_tag = pm.find(["description", "Description"])
        if desc_tag and desc_tag.get_text():
            desc_soup = BeautifulSoup(desc_tag.get_text(), "html.parser")
            tds = [t.get_text(strip=True) for t in desc_soup.find_all("td")]
            for i in range(0, len(tds), 2):
                clave = tds[i].upper()
                valor = tds[i + 1] if (i + 1) < len(tds) else ""
                if clave == "ZONA": zona_val = valor
                elif clave == "MACROZONA": macrozona_val = valor

        zona_final = zona_val or nombre
        polygons_for_pm = []
        
        for ct in pm.find_all(["coordinates", "Coordinates"]):
            if ct.get_text():
                coords = _parse_coords_string(ct.get_text())
                if coords: polygons_for_pm.append(coords)

        if polygons_for_pm:
            zonas.append({"zona": zona_final, "macrozona": macrozona_val, "polygons": polygons_for_pm})

    return pd.DataFrame(zonas)

# ============================================================
# ================= LÓGICA GEOMÉTRICA =======================
# ============================================================

def punto_en_poligono(lat, lon, poly) -> bool:
    dentro = False
    n = len(poly)
    for i in range(n):
        x1, y1 = poly[i][1], poly[i][0]
        x2, y2 = poly[(i + 1) % n][1], poly[(i + 1) % n][0]
        if ((y1 > lat) != (y2 > lat)) and (lon < (x2 - x1) * (lat - y1) / (y2 - y1 + 1e-12) + x1):
            dentro = not dentro
    return dentro

def asociar_paraderos_a_zonas(df_paraderos: pd.DataFrame, df_zonas: pd.DataFrame) -> pd.DataFrame:
    if df_zonas.empty or df_paraderos.empty: return df_paraderos
    
    print(f"🔄 Calculando zonas para {len(df_paraderos)} paraderos...")
    zona_out, macrozona_out = [], []

    for _, par in df_paraderos.iterrows():
        lat, lon = par["lat"], par["lon"]
        z_found, m_found = [], []
        for _, z in df_zonas.iterrows():
            if any(punto_en_poligono(lat, lon, poly) for poly in z["polygons"]):
                z_found.append(z["zona"])
                if z["macrozona"]: m_found.append(z["macrozona"])
                break 
        zona_out.append(", ".join(z_found) if z_found else "Fuera de zona")
        macrozona_out.append(", ".join(m_found) if m_found else "Fuera de zona")

    df_out = df_paraderos.copy()
    df_out["zona"] = zona_out
    df_out["macrozona"] = macrozona_out
    return df_out

def reordenar_paraderos_geograficamente(df_ruta, df_paraderos):
    """Ordena paraderos según proyección en la línea de ruta."""
    if df_ruta.empty or df_paraderos.empty: return df_paraderos

    print("📐 Calculando orden geográfico de los paraderos...")
    coords_ruta = list(zip(df_ruta['lon'], df_ruta['lat']))
    linea = LineString(coords_ruta)
    
    distancias = []
    for _, row in df_paraderos.iterrows():
        punto = Point(row['lon'], row['lat'])
        dist = linea.project(punto) 
        distancias.append(dist)
        
    df_out = df_paraderos.copy()
    df_out['distancia_desde_inicio'] = distancias
    df_out = df_out.sort_values('distancia_desde_inicio')
    df_out['nuevo_numero'] = range(1, len(df_out) + 1)
    
    return df_out



def filtrar_paraderos_cercanos_a_ruta(df_ruta, df_paraderos, max_dist=10):
    """
    max_dist = distancia máxima en METROS para considerar que la ruta 'pasa' por el paradero.
    """
    if df_ruta.empty or df_paraderos.empty:
        return df_paraderos

    linea = LineString(zip(df_ruta['lon'], df_ruta['lat']))

    paraderos_validos = []
    for _, row in df_paraderos.iterrows():
        punto = Point(row['lon'], row['lat'])
        dist = linea.distance(punto) * 111_139  # grados → metros

        if dist <= max_dist:
            paraderos_validos.append(row)  # Ruta pasa suficientemente cerca

    return pd.DataFrame(paraderos_validos)



# ============================================================
# ============ CARGAR RUTAS (PARSER ESTRUCTURADO) ============
# ============================================================

def cargar_kmz_estructurado(archivo_kmz: str):
    print(f"🚌 Cargando rutas desde: {archivo_kmz}...")
    data_rutas = []
    data_paraderos = []

    try:
        with zipfile.ZipFile(archivo_kmz, 'r') as z:
            kml_filename = [f for f in z.namelist() if f.endswith('.kml')][0]
            root = ET.fromstring(_clean_kml_text(z.read(kml_filename)))
    except Exception as e:
        print(f"❌ Error leyendo Rutas: {e}")
        return pd.DataFrame(), pd.DataFrame()

    for folder in root.findall(".//Folder"):
        folder_name = folder.findtext("name", "").strip()
        if "route stops labels" in folder_name.lower(): continue 
        placemarks = folder.findall("Placemark")
        if not placemarks: continue

        route_id = folder_name
        
        # A) BUSCAR LÍNEA DE RUTA
        rutas_candidatas = []
        for pm in placemarks:
            ls = pm.find("LineString")
            if ls is not None:
                nombre_ruta = pm.findtext("name", "").strip()
                coords = ls.findtext("coordinates", "")
                rutas_candidatas.append((nombre_ruta, coords))
        
        coords_final = None
        nombre_elegido = "NINGUNO"

        if rutas_candidatas:
            tiene_basura = len(rutas_candidatas) > 1
            if tiene_basura:
                print(f"\n🧐 Conflicto en '{folder_name}': {[r[0] for r in rutas_candidatas]}")

            match_tuple = next((x for x in rutas_candidatas if x[0] == folder_name), None)
            if not match_tuple:
                match_tuple = next((x for x in rutas_candidatas if "desvío" not in x[0].lower() and "regreso" not in x[0].lower()), None)
            if not match_tuple:
                match_tuple = rutas_candidatas[0]

            nombre_elegido, coords_final = match_tuple

            if tiene_basura:
                print(f"   ✅ GANADOR: '{nombre_elegido}'")

            if coords_final:
                puntos = _parse_coords_string(coords_final)
                for i, (lat, lon) in enumerate(puntos):
                    data_rutas.append({"route_id": route_id, "orden": i, "lat": lat, "lon": lon})

        # B) BUSCAR PARADEROS
        for pm in placemarks:
            pt = pm.find("Point")
            if pt is not None:
                coords_txt = pt.findtext("coordinates", "")
                pts = _parse_coords_string(coords_txt)
                if pts:
                    data_paraderos.append({
                        "route_id": route_id,
                        "nombre_paradero": pm.findtext("name", "").strip(),
                        "lat": pts[0][0], 
                        "lon": pts[0][1]
                    })

    return pd.DataFrame(data_rutas), pd.DataFrame(data_paraderos)


# ============================================================
# ========================= EJECUCIÓN ========================
# ============================================================

# 1. Cargar Datos Crudos del KMZ
df_rutas_raw, df_paraderos_raw = cargar_kmz_estructurado(ARCHIVO_RUTAS_KMZ)

# 2. Filtrar Ruta Seleccionada
print(f"\n🔍 Filtrando por: '{nombre_busqueda}' ...")

df_ruta_sel = df_rutas_raw[df_rutas_raw['route_id'].str.contains(nombre_busqueda, case=False, na=False)].copy()
df_paraderos_sel = df_paraderos_raw[df_paraderos_raw['route_id'].str.contains(nombre_busqueda, case=False, na=False)].copy()

if df_ruta_sel.empty:
    print(f"⚠️ No se encontró la ruta '{nombre_busqueda}'.")
else:
    ruta_id_actual = df_ruta_sel.iloc[0]["route_id"]
    print(f"✅ Trabajando sobre ruta: {ruta_id_actual}")

    # 3. Cargar e Integrar Paraderos del Excel
    df_manuales = cargar_paraderos_excel(ARCHIVO_NUEVOS, ruta_id_actual)
    df_manuales = filtrar_paraderos_cercanos_a_ruta(df_ruta_sel, df_manuales, max_dist=20)
    if not df_manuales.empty:
        df_paraderos_sel = pd.concat([df_paraderos_sel, df_manuales], ignore_index=True)

    # 4. Reordenamiento Geográfico Inteligente
    try:
        df_paraderos_sel = reordenar_paraderos_geograficamente(df_ruta_sel, df_paraderos_sel)
        print("✅ Paraderos reordenados correctamente por posición en la ruta.")
        # print(df_paraderos_sel[['nuevo_numero', 'nombre_paradero']].to_string())
    except Exception as e:
        print(f"⚠️ Error en reordenamiento geográfico: {e}")

    # 5. Cargar Zonas y Asociar
    print(f"📊 Total paraderos a analizar: {len(df_paraderos_sel)}")
    df_zonas = cargar_zonas_kmz(ARCHIVO_ZONAS_KMZ)
    df_paraderos_sel = asociar_paraderos_a_zonas(df_paraderos_sel, df_zonas)
# 2. Filtrar Ruta Seleccionada
print(f"\n🔍 Filtrando por: '{nombre_busqueda}' ...")

df_ruta_sel = df_rutas_raw[df_rutas_raw['route_id'].str.contains(nombre_busqueda, case=False, na=False)].copy()

if df_ruta_sel.empty:
    print(f"⚠️ No se encontró la ruta '{nombre_busqueda}'.")
else:
    ruta_id_actual = df_ruta_sel.iloc[0]["route_id"]
    print(f"✅ Trabajando sobre ruta: {ruta_id_actual}")

    # ============================================================
    # NUEVO: Cargar paraderos desde paraderos_ruta_{nombre}.xlsx
    # ============================================================
    archivo_paraderos_excel = f"paraderos_ruta_{nombre_busqueda}.xlsx"

    print(f"📥 Cargando paraderos desde: {archivo_paraderos_excel} ...")
    try:
        df_paraderos_sel = pd.read_excel(archivo_paraderos_excel)

        # Limpieza mínima
        df_paraderos_sel['lat'] = pd.to_numeric(df_paraderos_sel['lat'], errors='coerce')
        df_paraderos_sel['lon'] = pd.to_numeric(df_paraderos_sel['lon'], errors='coerce')
        df_paraderos_sel = df_paraderos_sel.dropna(subset=['lat','lon'])

        print(f"   ✅ {len(df_paraderos_sel)} paraderos cargados desde el archivo local.")

    except Exception as e:
        print(f"❌ No se pudo cargar el archivo '{archivo_paraderos_excel}': {e}")
        print("⚠️ Usando paraderos del KMZ como respaldo.")
        
        df_paraderos_sel = df_paraderos_raw[df_paraderos_raw['route_id'].str.contains(nombre_busqueda, case=False, na=False)].copy()

    # Estos paraderos reemplazan por completo a los originales
    df_paraderos_sel['route_id'] = ruta_id_actual

























    # ============================================================
    # ======================= MAPA FINAL =========================
    # ============================================================
    print("\n🗺️ Generando mapa...")
    centro = [df_ruta_sel.iloc[0]["lat"], df_ruta_sel.iloc[0]["lon"]]
    m = folium.Map(location=centro, zoom_start=14)

    # A) Dibujar Zonas
    for _, z in df_zonas.iterrows():
        for poly in z["polygons"]:
            folium.Polygon(locations=poly, fill=True, color="#999999", weight=1, fill_opacity=0.1,
                tooltip=f"{z['zona']} | {z['macrozona']}").add_to(m)

    # B) Dibujar Ruta
    folium.PolyLine(df_ruta_sel[['lat', 'lon']].values.tolist(), color="blue", weight=5, opacity=0.8).add_to(m)

    # C) Dibujar Paraderos (Verde = Nuevos, Rojo = Originales)
    for _, p in df_paraderos_sel.iterrows():
        es_nuevo = "Paradero Manual" in str(p['nombre_paradero']) or "Agregado" in str(p['nombre_paradero'])
        color_icono = "green" if es_nuevo else "red"
        icono_tipo = "plus" if es_nuevo else "bus"
        
        html_popup = f"""
        <div style="font-family: Arial; width: 150px;">
            <b>{'🆕 ' if es_nuevo else '🚏 '}{p['nombre_paradero']}</b><br>
            <hr style="margin: 5px 0;">
            <b>Zona:</b> {p['zona']}<br>
            <b>Macrozona:</b> {p['macrozona']}
        </div>
        """
        folium.Marker(
            location=[p["lat"], p["lon"]],
            popup=folium.Popup(html_popup, max_width=200),
            icon=folium.Icon(color=color_icono, icon=icono_tipo, prefix="fa")
        ).add_to(m)

    nombre_salida = f"mapa_{nombre_busqueda}_actualizado.html"
    m.save(nombre_salida)
    print(f"🎉 Mapa guardado como: {nombre_salida}")
    display(m)

🚌 Cargando rutas desde: Recorrido Ruta Las Galaxias L80 (1) (2).kmz...

🧐 Conflicto en '80LZ_I': ['80LZ_I', 'Galaxias (Regreso Valle)', 'Galaxias (Desvío a Ccp)']
   ✅ GANADOR: '80LZ_I'

🔍 Filtrando por: '80JQ_I' ...
✅ Trabajando sobre ruta: 80JQ_I
📥 Cargando paraderos extra desde: nuevos_paraderos.xlsx...
🔧 Detectado formato pegado de Google Maps. Separando...
✅ Se agregaron 9 paraderos manuales a la ruta 80JQ_I.
📐 Calculando orden geográfico de los paraderos...
✅ Paraderos reordenados correctamente por posición en la ruta.
📊 Total paraderos a analizar: 27
🌍 Cargando zonas desde: zonificacion.kmz...
🔄 Calculando zonas para 27 paraderos...

🔍 Filtrando por: '80JQ_I' ...
✅ Trabajando sobre ruta: 80JQ_I
📥 Cargando paraderos desde: paraderos_ruta_80JQ_I.xlsx ...
   ✅ 129 paraderos cargados desde el archivo local.

🗺️ Generando mapa...


KeyError: 'zona'

In [ ]:
import requests
import time
import pandas as pd
import winsound
from IPython.display import clear_output  # limpiar pantalla

def obtener_paraderos_osm(df_ruta, radio_metros=120):
    """
    Recorre la ruta y consulta Overpass para obtener paraderos cercanos.
    Devuelve DataFrame con paraderos únicos.
    """

    overpass_url = "https://overpass-api.de/api/interpreter"
    paraderos = []
    vistos = set()

    total_puntos = len(df_ruta)

    ultimo_paradero = "Ninguno aún"

    try:
        for actual, (idx, row) in enumerate(df_ruta.iterrows(), start=1):

            clear_output(wait=True)

            lat, lon = row["lat"], row["lon"]

            print("🔎 Buscando paraderos en OSM...")
            print(f"➡ Procesando punto {actual}/{total_puntos} (idx={idx})")
            print(f"📊 Paraderos acumulados: {len(paraderos)}")
            print(f"🆕 Último paradero encontrado: {ultimo_paradero}\n")

            query = f"""
            [out:json][timeout:25];
            (
              node(around:{radio_metros}, {lat}, {lon})["highway"="bus_stop"];
              node(around:{radio_metros}, {lat}, {lon})["public_transport"="platform"];
            );
            out body;
            """

            encontrados_en_punto = 0

            try:
                r = requests.post(overpass_url, data={"data": query})
                datos = r.json()

                if "elements" in datos:
                    for e in datos["elements"]:
                        pid = e["id"]
                        if pid in vistos:
                            continue

                        vistos.add(pid)
                        encontrados_en_punto += 1

                        # Registrar nombre
                        nombre = e["tags"].get("name", "Paradero OSM sin nombre")
                        ultimo_paradero = nombre  # <<< mostrar en pantalla

                        plat = {
                            "route_id": nombre_busqueda,
                            "nombre_paradero": nombre,
                            "lat": e["lat"],
                            "lon": e["lon"]
                        }
                        paraderos.append(plat)

                print(f"✔ Paraderos en este punto: {encontrados_en_punto}")
                print(f"📌 Total acumulado: {len(paraderos)}")
                print(f"🕒 Último paradero en este punto: {ultimo_paradero}")

                time.sleep(1.0)

            except Exception as ex:
                print(f"⚠️ Error en punto idx={idx}: {ex}")
                time.sleep(1)
                continue

    except KeyboardInterrupt:
        clear_output(wait=True)
        print("🛑 Búsqueda detenida manualmente.")
        print(f"📁 Paraderos guardados hasta ahora: {len(paraderos)}")
        print(f"🆕 Último paradero encontrado: {ultimo_paradero}")
        return pd.DataFrame(paraderos)

    clear_output(wait=True)
    print(f"✅ Búsqueda completada. Paraderos totales: {len(paraderos)}")
    print(f"🆕 Último paradero encontrado: {ultimo_paradero}")

    return pd.DataFrame(paraderos)


# Ejecutar búsqueda
df_osm = obtener_paraderos_osm(df_ruta_sel, radio_metros=25)

# Timbre final
winsound.Beep(1000, 700)
winsound.Beep(1200, 700)
winsound.Beep(1500, 700)


🛑 Búsqueda detenida manualmente.
📁 Paraderos guardados hasta ahora: 2
🆕 Último paradero encontrado: Terminal de Buses Las Galaxias / Oriente


In [ ]:
df_osm.to_excel("paraderos_ruta_{nombre_busqueda}.xlsx", index=False)
print(f"Archivo guardado como paraderos_ruta_{nombre_busqueda}.xlsx")


Archivo guardado como paraderos_ruta80LZ_I.xlsx


In [ ]:
import folium

# ================================================
# ========== MAPA SOLO CON df_osm =================
# ================================================

# Si df_osm está vacío, avisa y no genera mapa
if df_osm.empty:
    print("⚠️ df_osm está vacío, no hay paraderos para mostrar.")
else:
    # Centro del mapa = primer paradero
    centro = [df_osm.iloc[0]["lat"], df_osm.iloc[0]["lon"]]
    m_osm = folium.Map(location=centro, zoom_start=15)

    # Agregar los paraderos encontrados
    for _, p in df_osm.iterrows():
        folium.Marker(
            location=[p["lat"], p["lon"]],
            tooltip=p.get("nombre_paradero", "Paradero"),
            icon=folium.Icon(color="red", icon="bus", prefix="fa")
        ).add_to(m_osm)

    # Guardar mapa
    m_osm.save("mapa_paraderos_osm.html")
    print("🗺️ Mapa generado: mapa_paraderos_osm.html")

    # Mostrar dentro de Jupyter
    display(m_osm)


🗺️ Mapa generado: mapa_paraderos_osm.html
